
# Nome: Raylander Marques Melo
# Matrícula: 494563

In [15]:
!pip install numpy scipy pandas scikit-learn spacy emoji sentence-transformers unidecode


In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
import emoji
import spacy
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.base import TransformerMixin
import unidecode
import re

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC  # Import do SVC adicionado


/home/raylander/Desktop/Introdução a Ciência de Dados/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Crie um arquivo Jupyter Notebook e realize as seguintes operações:

## a) Ler o dataset fakeTelegram.BR_2022.csv, o qual está disponível no link a seguir: https://drive.google.com/file/d/1c_hLzk85pYw-huHSnFYZM_gn-dUsYRDm/view?usp=drive_link

In [2]:
# Abre do aquivo CSV e printa alguns elementos
df = pd.read_csv("/home/raylander/Desktop/Introdução a Ciência de Dados/fakeTelegram.BR_2022_tratado.csv")
df

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,...,id_message,message_type,messenger,media_name,media_md5,caracteres,words,viral,sharings,sentiment
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,...,16385,Texto,telegram,NaN,NaN,110,20,0,1.0,0
1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",...,16386,Texto,telegram,NaN,NaN,141,23,0,1.0,1
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",...,16366,Texto,telegram,NaN,NaN,350,59,0,1.0,-1
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,...,19281,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb,40,7,0,1.0,0
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,False,NaN,...,507185,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637,0,0,1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557581,2022-11-11 12:06:15,333e9869f23dbd4682d1be382d9c1e59,e56ec342fc599ebb4ed89655eb6f03aa,25e43b6a58b848c43ad5b5f9e979822a.jpg,url,https://terrabrasilnoticias.com/2022/11/bndes-...,True,True,False,"BNDES tem lucro de R$ 9,6 bilhões no terceiro ...",...,575796,Url,telegram,NaN,25e43b6a58b848c43ad5b5f9e979822a,152,12,1,2.0,1
557582,2022-11-11 12:09:08,NaN,5b10d7739171149be6d9961e3350c071,657949d03e4088f6b332e2686ccd3221.jpg,url,https://youtu.be/8g1Vz9_0xVk,True,True,False,https://youtu.be/8g1Vz9_0xVk,...,1286443,Url,telegram,NaN,657949d03e4088f6b332e2686ccd3221,28,1,1,2.0,0
557583,2022-11-11 12:09:47,NaN,1590a03f43b5ba4b6147a1c5e1dd357b,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro.\n\n...",...,13294,Imagem,telegram,NaN,a21848a61045380a6483866daed0ca0e,68,6,1,4.0,0
557584,2022-11-11 12:09:46,NaN,5b10d7739171149be6d9961e3350c071,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro.\n\n...",...,1286444,Imagem,telegram,NaN,a21848a61045380a6483866daed0ca0e,68,6,1,4.0,0


## b)  Remova os trava-zaps, as linhas repetidas (duplicadas) e textos com menos de 5 palavras. 

In [3]:
# Soma a quantidade de elementos com True na coluna 'trava_zap' já existente no dataset
print(df['trava_zap'].sum())

16


In [4]:
# Retira as linhas que possuem trava-zap
df = df[df['trava_zap'] == False]
# Printa o dataset resultante
df

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,...,id_message,message_type,messenger,media_name,media_md5,caracteres,words,viral,sharings,sentiment
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,...,16385,Texto,telegram,NaN,NaN,110,20,0,1.0,0
1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",...,16386,Texto,telegram,NaN,NaN,141,23,0,1.0,1
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",...,16366,Texto,telegram,NaN,NaN,350,59,0,1.0,-1
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,...,19281,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb,40,7,0,1.0,0
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,False,NaN,...,507185,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637,0,0,1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557581,2022-11-11 12:06:15,333e9869f23dbd4682d1be382d9c1e59,e56ec342fc599ebb4ed89655eb6f03aa,25e43b6a58b848c43ad5b5f9e979822a.jpg,url,https://terrabrasilnoticias.com/2022/11/bndes-...,True,True,False,"BNDES tem lucro de R$ 9,6 bilhões no terceiro ...",...,575796,Url,telegram,NaN,25e43b6a58b848c43ad5b5f9e979822a,152,12,1,2.0,1
557582,2022-11-11 12:09:08,NaN,5b10d7739171149be6d9961e3350c071,657949d03e4088f6b332e2686ccd3221.jpg,url,https://youtu.be/8g1Vz9_0xVk,True,True,False,https://youtu.be/8g1Vz9_0xVk,...,1286443,Url,telegram,NaN,657949d03e4088f6b332e2686ccd3221,28,1,1,2.0,0
557583,2022-11-11 12:09:47,NaN,1590a03f43b5ba4b6147a1c5e1dd357b,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro.\n\n...",...,13294,Imagem,telegram,NaN,a21848a61045380a6483866daed0ca0e,68,6,1,4.0,0
557584,2022-11-11 12:09:46,NaN,5b10d7739171149be6d9961e3350c071,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro.\n\n...",...,1286444,Imagem,telegram,NaN,a21848a61045380a6483866daed0ca0e,68,6,1,4.0,0


In [5]:
df

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,...,id_message,message_type,messenger,media_name,media_md5,caracteres,words,viral,sharings,sentiment
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,...,16385,Texto,telegram,NaN,NaN,110,20,0,1.0,0
1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",...,16386,Texto,telegram,NaN,NaN,141,23,0,1.0,1
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",...,16366,Texto,telegram,NaN,NaN,350,59,0,1.0,-1
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,...,19281,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb,40,7,0,1.0,0
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,False,NaN,...,507185,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637,0,0,1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557581,2022-11-11 12:06:15,333e9869f23dbd4682d1be382d9c1e59,e56ec342fc599ebb4ed89655eb6f03aa,25e43b6a58b848c43ad5b5f9e979822a.jpg,url,https://terrabrasilnoticias.com/2022/11/bndes-...,True,True,False,"BNDES tem lucro de R$ 9,6 bilhões no terceiro ...",...,575796,Url,telegram,NaN,25e43b6a58b848c43ad5b5f9e979822a,152,12,1,2.0,1
557582,2022-11-11 12:09:08,NaN,5b10d7739171149be6d9961e3350c071,657949d03e4088f6b332e2686ccd3221.jpg,url,https://youtu.be/8g1Vz9_0xVk,True,True,False,https://youtu.be/8g1Vz9_0xVk,...,1286443,Url,telegram,NaN,657949d03e4088f6b332e2686ccd3221,28,1,1,2.0,0
557583,2022-11-11 12:09:47,NaN,1590a03f43b5ba4b6147a1c5e1dd357b,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro.\n\n...",...,13294,Imagem,telegram,NaN,a21848a61045380a6483866daed0ca0e,68,6,1,4.0,0
557584,2022-11-11 12:09:46,NaN,5b10d7739171149be6d9961e3350c071,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro.\n\n...",...,1286444,Imagem,telegram,NaN,a21848a61045380a6483866daed0ca0e,68,6,1,4.0,0


In [6]:
# Remover linhas duplicadas com base em todas as colunas
df = df.drop_duplicates()
df

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,...,id_message,message_type,messenger,media_name,media_md5,caracteres,words,viral,sharings,sentiment
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,...,16385,Texto,telegram,NaN,NaN,110,20,0,1.0,0
1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",...,16386,Texto,telegram,NaN,NaN,141,23,0,1.0,1
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",...,16366,Texto,telegram,NaN,NaN,350,59,0,1.0,-1
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,...,19281,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb,40,7,0,1.0,0
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,False,NaN,...,507185,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637,0,0,1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557581,2022-11-11 12:06:15,333e9869f23dbd4682d1be382d9c1e59,e56ec342fc599ebb4ed89655eb6f03aa,25e43b6a58b848c43ad5b5f9e979822a.jpg,url,https://terrabrasilnoticias.com/2022/11/bndes-...,True,True,False,"BNDES tem lucro de R$ 9,6 bilhões no terceiro ...",...,575796,Url,telegram,NaN,25e43b6a58b848c43ad5b5f9e979822a,152,12,1,2.0,1
557582,2022-11-11 12:09:08,NaN,5b10d7739171149be6d9961e3350c071,657949d03e4088f6b332e2686ccd3221.jpg,url,https://youtu.be/8g1Vz9_0xVk,True,True,False,https://youtu.be/8g1Vz9_0xVk,...,1286443,Url,telegram,NaN,657949d03e4088f6b332e2686ccd3221,28,1,1,2.0,0
557583,2022-11-11 12:09:47,NaN,1590a03f43b5ba4b6147a1c5e1dd357b,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro.\n\n...",...,13294,Imagem,telegram,NaN,a21848a61045380a6483866daed0ca0e,68,6,1,4.0,0
557584,2022-11-11 12:09:46,NaN,5b10d7739171149be6d9961e3350c071,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro.\n\n...",...,1286444,Imagem,telegram,NaN,a21848a61045380a6483866daed0ca0e,68,6,1,4.0,0


In [7]:
# Função para contar o número de palavras
def count_words(text):
    if isinstance(text, str):  # Verifica se o valor é uma string
        return len(text.split())
    return 0  # Caso não seja string, retorna 0 palavras

# Adiciona a coluna 'word_count' ao DataFrame
df['word_count'] = df['text_content_anonymous'].apply(count_words)

# Filtra os textos com mais de 5 palavras
df = df[df['word_count'] >= 5]
df


,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,...,message_type,messenger,media_name,media_md5,caracteres,words,viral,sharings,sentiment,word_count
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,...,Texto,telegram,NaN,NaN,110,20,0,1.0,0,20
1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",...,Texto,telegram,NaN,NaN,141,23,0,1.0,1,23
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",...,Texto,telegram,NaN,NaN,350,59,0,1.0,-1,59
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,...,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb,40,7,0,1.0,0,7
5,2022-10-05 06:28:30,NaN,b52442a5fbc459ae590dca0d215e32f9,NaN,NaN,NaN,False,False,False,Kķkkkkk to rindo até agora....Quem disse q ia ...,...,Texto,telegram,NaN,NaN,133,25,0,1.0,1,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557580,2022-11-11 12:05:41,3e49fd40fd973ee1b8f1a6d58feb4a54,4c6519d965020abc048521dfa837b9bb,9dd3b338a07226a04f97df82fbbcb36b.jpg,image/jpg,https://t.me/Manager_Marcus_Chat_Up_Now,True,True,False,NOVOS MEMBROS SEMPRE CLICAM NO LINK DO ADMINIS...,...,Imagem,telegram,NaN,9dd3b338a07226a04f97df82fbbcb36b,254,28,1,12.0,0,28
557581,2022-11-11 12:06:15,333e9869f23dbd4682d1be382d9c1e59,e56ec342fc599ebb4ed89655eb6f03aa,25e43b6a58b848c43ad5b5f9e979822a.jpg,url,https://terrabrasilnoticias.com/2022/11/bndes-...,True,True,False,"BNDES tem lucro de R$ 9,6 bilhões no terceiro ...",...,Url,telegram,NaN,25e43b6a58b848c43ad5b5f9e979822a,152,12,1,2.0,1,12
557583,2022-11-11 12:09:47,NaN,1590a03f43b5ba4b6147a1c5e1dd357b,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro.\n\n...",...,Imagem,telegram,NaN,a21848a61045380a6483866daed0ca0e,68,6,1,4.0,0,6
557584,2022-11-11 12:09:46,NaN,5b10d7739171149be6d9961e3350c071,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro.\n\n...",...,Imagem,telegram,NaN,a21848a61045380a6483866daed0ca0e,68,6,1,4.0,0,6


In [8]:
df.columns

Index(['date_message', 'id_member_anonymous', 'id_group_anonymous', 'media',
       'media_type', 'media_url', 'has_media', 'has_media_url', 'trava_zap',
       'text_content_anonymous', 'dataset_info_id', 'date_system',
       'score_sentiment', 'score_misinformation', 'id_message', 'message_type',
       'messenger', 'media_name', 'media_md5', 'caracteres', 'words', 'viral',
       'sharings', 'sentiment', 'word_count'],
      dtype='object')

## c)  Agrupe as linhas com postagens iguais ou extremamente semelhantes. Aqui você pode  utilizar  uma  métrica  de  semelhança  de  textos.  Crie  uma  variável  para representar a quantidade de vezes que a mensagem foi compartilhada.  Observe que ao agrupar linhas que possuem a “mesma” postagem (texto), você deve escolher como valor para as variáveis data e hora da postagem, os valores da cópia mais antiga.

In [ ]:
# Carrega o modelo de embeddings
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # Rápido e eficiente

df = df.reset_index(drop=True)  # Garante índice limpo


df.loc[:, 'text'] = df['text_content_anonymous'].fillna('').str.lower()

# Calcula os embeddings
embeddings = model.encode(df['text'], show_progress_bar=True)

# Salva os embeddings como array numpy
np.save('embeddings.npy', embeddings)


Batches:   0%|          | 0/10603 [00:00<?, ?it/s]

In [66]:
# Carrega os embeddings salvos
embeddings = np.load('embeddings.npy')

In [ ]:
# Agrupamento com DBSCAN baseado em similaridade de cosseno
clustering = DBSCAN(eps=0.1, min_samples=5, metric='cosine').fit(embeddings)
df['cluster'] = clustering.labels_

In [83]:
# Agrupar mensagens semelhantes (clusters com mesmo ID)
agrupado = df[df['cluster'] != -1].groupby('cluster').agg({
    'text_content_anonymous': 'first',  # mantém o texto representativo
    'date_message': 'min',  # pega a data mais antiga
    'sharings': 'sum',  # soma dos compartilhamentos
    'id_message': 'count'  # quantidade de mensagens no cluster
}).rename(columns={
    'text_content_anonymous': 'texto_base',
    'date_message': 'data_mais_antiga',
    'sharings': 'total_shares',
    'id_message': 'quantidade_replicas'
}).reset_index()

# Resultados
print(agrupado.head())

   cluster                                         texto_base  \
0        0  *SE ALGUÉM TE PERGUNTAR O QUE FOI QUE BOLSONAR...   
1        1  O Deputado Federal pelo NOVO e que foi candida...   
2        2  Saiam desse grupo amigos bolsonaristas, urgent...   
3        3  🚨Olá SEJA bem vindo(a) ..- ao grupo Especuland...   
4        4  Feedback de Gabriel De Sousa 24 anos. Brazil, ...   

      data_mais_antiga  total_shares  quantidade_replicas  
0  2022-09-30 00:50:48         849.0                   89  
1  2022-10-01 20:49:53          79.0                   51  
2  2022-10-05 02:25:57         469.0                   43  
3  2022-09-29 01:02:26        1902.0                 1500  
4  2022-09-29 03:02:20       78639.0                 6353  


In [84]:
agrupado

,cluster,texto_base,data_mais_antiga,total_shares,quantidade_replicas
0,0,*SE ALGUÉM TE PERGUNTAR O QUE FOI QUE BOLSONAR...,2022-09-30 00:50:48,849.0,89
1,1,O Deputado Federal pelo NOVO e que foi candida...,2022-10-01 20:49:53,79.0,51
2,2,"Saiam desse grupo amigos bolsonaristas, urgent...",2022-10-05 02:25:57,469.0,43
3,3,🚨Olá SEJA bem vindo(a) ..- ao grupo Especuland...,2022-09-29 01:02:26,1902.0,1500
4,4,"Feedback de Gabriel De Sousa 24 anos. Brazil, ...",2022-09-29 03:02:20,78639.0,6353
...,...,...,...,...,...
6542,6542,Entendeu porque o relatório saiu inconclusivo?...,2022-11-10 00:18:34,17.0,5
6543,6543,"Karina Michelin, conseguiu subir esse vídeo, p...",2022-11-10 11:38:07,18.0,8
6544,6544,Tiraram do YouTube mas tá rolando a live ainda...,2022-11-06 16:07:25,144.0,12
6545,6545,Liberação do aborto ou direito à vida? Liberaç...,2022-10-24 21:31:39,12.0,6


In [87]:
# Mensagens isoladas (sem similaridade suficiente)
nao_agrupadas = df[df['cluster'] == -1].copy()
nao_agrupadas['texto_base'] = nao_agrupadas['text_content_anonymous']
nao_agrupadas['data_mais_antiga'] = nao_agrupadas['date_message']
nao_agrupadas['total_shares'] = nao_agrupadas['sharings']
nao_agrupadas['quantidade_replicas'] = 1  # cada uma é uma "réplica única"

# Selecionar colunas relevantes para uniformizar com 'agrupado'
nao_agrupadas = nao_agrupadas[[
    'cluster', 'texto_base', 'data_mais_antiga', 'total_shares', 'quantidade_replicas'
]]

# Unir os dois DataFrames (agrupados + não agrupados)
df_final = pd.concat([agrupado, nao_agrupadas], ignore_index=True)

In [88]:
df_final

,cluster,texto_base,data_mais_antiga,total_shares,quantidade_replicas
0,0,*SE ALGUÉM TE PERGUNTAR O QUE FOI QUE BOLSONAR...,2022-09-30 00:50:48,849.0,89
1,1,O Deputado Federal pelo NOVO e que foi candida...,2022-10-01 20:49:53,79.0,51
2,2,"Saiam desse grupo amigos bolsonaristas, urgent...",2022-10-05 02:25:57,469.0,43
3,3,🚨Olá SEJA bem vindo(a) ..- ao grupo Especuland...,2022-09-29 01:02:26,1902.0,1500
4,4,"Feedback de Gabriel De Sousa 24 anos. Brazil, ...",2022-09-29 03:02:20,78639.0,6353
...,...,...,...,...,...
213899,-1,"Deputado é reeleito nos Estados Unidos, mas mo...",2022-11-11 12:05:38,3.0,1
213900,-1,"BNDES tem lucro de R$ 9,6 bilhões no terceiro ...",2022-11-11 12:06:15,2.0,1
213901,-1,"Empresários, demitam os petistas primeiro.\n\n...",2022-11-11 12:09:47,4.0,1
213902,-1,"Empresários, demitam os petistas primeiro.\n\n...",2022-11-11 12:09:46,4.0,1


In [89]:
df_final.to_csv('./df_final.csv', index=False)

## d)  Você pode criar novos atributos numéricos, tais como: quantidade de palavras, quantidade de caracteres etc.

#### execute o seguinte comando no terminal: "python -m spacy download pt_core_news_sm"

In [5]:
agrupado = pd.read_csv('./df_final.csv')

In [6]:
# Carregue o modelo de português
nlp = spacy.load("pt_core_news_sm")

# Função para contar sentenças
def contar_sentencas(texto):
    doc = nlp(texto)
    return len(list(doc.sents))

# Agora sua função para contar emojis
def count_emojis(text):
    return len([c for c in text if c in emoji.EMOJI_DATA])

# Aplique os atributos no seu DataFrame agrupado
agrupado['num_sentences'] = agrupado['texto_base'].apply(contar_sentencas)
agrupado['num_uppercase_words'] = agrupado['texto_base'].apply(lambda x: sum(1 for w in x.split() if w.isupper()))
agrupado['num_exclamations'] = agrupado['texto_base'].str.count('!')
agrupado['num_questions'] = agrupado['texto_base'].str.count(r'\?')
agrupado['num_digits'] = agrupado['texto_base'].apply(lambda x: sum(c.isdigit() for c in x))
agrupado['num_links'] = agrupado['texto_base'].str.count(r'http[s]?://\S+')
agrupado['num_emojis'] = agrupado['texto_base'].apply(count_emojis)
agrupado['num_hashtags'] = agrupado['texto_base'].str.count(r'#\w+')
agrupado['num_mentions'] = agrupado['texto_base'].str.count(r'@\w+')
agrupado['char_count'] = agrupado['texto_base'].str.len()
agrupado['word_count'] = agrupado['texto_base'].str.split().str.len()
agrupado['unique_word_count'] = agrupado['texto_base'].apply(lambda x: len(set(x.split())))
agrupado['avg_word_length'] = agrupado['char_count'] / agrupado['word_count']


In [7]:
# Calcular a mediana e o desvio padrão da coluna 'total_shares'
median_shares = agrupado['total_shares'].median()
std_shares = agrupado['total_shares'].std()

# Definir o limiar como mediana + 2 * desvio padrão
# threshold = median_shares + 2 * std_shares
threshold = median_shares

# Criar a coluna 'viral' baseada no limiar
agrupado['viral'] = np.where(agrupado['total_shares'] > threshold, 1, 0)

# Verificar a distribuição da nova coluna
print(agrupado['viral'].value_counts())

viral
0    159490
1     54414
Name: count, dtype: int64


In [10]:
# Calculando os quartis
quartis = agrupado['total_shares'].quantile([0.25, 0.5, 0.75])

# Verificar se há quartis repetidos
if len(set(quartis)) < 3:  # Se houver valores duplicados
    # Ajustar os bins para valores únicos
    unique_bins = sorted(list({-1, *quartis.unique(), float('inf')}))
    
    # Ajustar labels para o número de intervalos resultantes
    num_bins = len(unique_bins) - 1
    labels = list(range(num_bins))  # [0, 1, 2] se tivermos 3 intervalos, etc.
    
    agrupado['nivel_viralidade'] = pd.cut(
        agrupado['total_shares'],
        bins=unique_bins,
        labels=labels,
        duplicates='drop'
    )
else:
    # Proceder normalmente se não houver duplicatas
    agrupado['nivel_viralidade'] = pd.cut(
        agrupado['total_shares'],
        bins=[-1, quartis[0.25], quartis[0.5], quartis[0.75], float('inf')],
        labels=[0, 1, 2, 3]
    )

In [11]:
agrupado['nivel_viralidade']

0         2
1         2
2         2
3         2
4         2
         ..
213899    2
213900    1
213901    2
213902    2
213903    2
Name: nivel_viralidade, Length: 213904, dtype: category
Categories (3, int64): [0 < 1 < 2]

In [12]:
agrupado.to_csv('./agrupado.csv', index=False)

In [14]:
agrupado

,cluster,texto_base,data_mais_antiga,total_shares,quantidade_replicas,num_sentences,num_uppercase_words,num_exclamations,num_questions,num_digits,num_links,num_emojis,num_hashtags,num_mentions,char_count,word_count,unique_word_count,avg_word_length,viral,nivel_viralidade
0,0,*SE ALGUÉM TE PERGUNTAR O QUE FOI QUE BOLSONAR...,2022-09-30 00:50:48,849.0,89,82,41,0,0,172,0,0,0,0,3669,618,399,5.936893,1,2
1,1,O Deputado Federal pelo NOVO e que foi candida...,2022-10-01 20:49:53,79.0,51,5,4,0,1,29,1,0,30,0,1242,153,118,8.117647,1,2
2,2,"Saiam desse grupo amigos bolsonaristas, urgent...",2022-10-05 02:25:57,469.0,43,16,80,7,0,0,0,0,0,0,1257,209,148,6.014354,1,2
3,3,🚨Olá SEJA bem vindo(a) ..- ao grupo Especuland...,2022-09-29 01:02:26,1902.0,1500,10,1,0,0,0,1,8,0,0,353,47,42,7.510638,1,2
4,4,"Feedback de Gabriel De Sousa 24 anos. Brazil, ...",2022-09-29 03:02:20,78639.0,6353,4,2,0,0,2,0,0,0,0,143,24,23,5.958333,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213899,-1,"Deputado é reeleito nos Estados Unidos, mas mo...",2022-11-11 12:05:38,3.0,1,1,0,0,0,8,1,0,0,0,162,13,13,12.461538,1,2
213900,-1,"BNDES tem lucro de R$ 9,6 bilhões no terceiro ...",2022-11-11 12:06:15,2.0,1,1,2,0,0,10,1,0,0,0,152,12,12,12.666667,1,1
213901,-1,"Empresários, demitam os petistas primeiro.\n\n...",2022-11-11 12:09:47,4.0,1,2,0,0,0,0,1,0,0,0,68,6,6,11.333333,1,2
213902,-1,"Empresários, demitam os petistas primeiro.\n\n...",2022-11-11 12:09:46,4.0,1,2,0,0,0,0,1,0,0,0,68,6,6,11.333333,1,2


# Utilizando  os  dados  referente  a  postagens  no  Telegram,  crie  um  modelo  preditivo (classificador multiclasse) para classificar uma mensagem em níveis de viralidade. Escolha uma estratégia para definir o número de níveis de viralidade (classes). Por exemplo, você definir quatro níveis de viralidade a partir dos quartis da quantidade de compartilhamentos. 

In [2]:
agrupado = pd.read_csv('./agrupado.csv')

In [3]:
print(agrupado.columns)

Index(['cluster', 'texto_base', 'data_mais_antiga', 'total_shares',
       'quantidade_replicas', 'num_sentences', 'num_uppercase_words',
       'num_exclamations', 'num_questions', 'num_digits', 'num_links',
       'num_emojis', 'num_hashtags', 'num_mentions', 'char_count',
       'word_count', 'unique_word_count', 'avg_word_length', 'viral',
       'nivel_viralidade'],
      dtype='object')


In [4]:
print(agrupado.dtypes)

cluster                  int64
texto_base              object
data_mais_antiga        object
total_shares           float64
quantidade_replicas      int64
num_sentences            int64
num_uppercase_words      int64
num_exclamations         int64
num_questions            int64
num_digits               int64
num_links                int64
num_emojis               int64
num_hashtags             int64
num_mentions             int64
char_count               int64
word_count               int64
unique_word_count        int64
avg_word_length        float64
viral                    int64
nivel_viralidade         int64
dtype: object


In [5]:
agrupado_clean = agrupado.dropna(subset=['texto_base', 'viral'])
# Features e target
X_text = agrupado_clean['texto_base']  # coluna com os textos
# X_text_amostra = X_text.sample(1000, random_state=42)  # amostra de 1000 textos
y = agrupado_clean['nivel_viralidade']     # target numérico

def normalizar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r"http\S+", "", texto)  # remove URLs
    texto = emoji.replace_emoji(texto, '')  # remove emojis
    texto = re.sub(r"[^a-zA-Z0-9\s]", "", texto)  # remove pontuação
    texto = unidecode.unidecode(texto)  # remove acentos
    texto = re.sub(r"\s+", " ", texto).strip()
    return texto

agrupado['texto_normalizado'] = agrupado_clean['texto_base'].apply(normalizar_texto)

X_text_norm = agrupado['texto_normalizado']  # coluna com os textos normalizados
# X_text_norm_amostra = X_text_norm.sample(1000, random_state=42)  # amostra de 1000 textos normalizados

# # Extrair features numéricas do DataFrame agrupado
# features_numericas = agrupado.drop(columns=['texto_base','total_shares','data_mais_antiga', 'cluster']) 

# # Biblioteca de normalização z-score
# scaler = StandardScaler()

# # Biblioteca de normalização MinMax
# scaler = MinMaxScaler()

# # Normalização das features numéricas
# X_num_minmax = scaler.fit_transform(features_numericas)
# X_num_zcore = scaler.fit_transform(features_numericas)  # Isso retorna um array denso (NumPy)

In [6]:
# Vetorização com TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 1))
X_tfidf_unigrama = vectorizer.fit_transform(X_text)
X_tfidf_unigrama_norm = vectorizer.fit_transform(X_text_norm)

vectorizer = TfidfVectorizer(ngram_range=(2, 2))
X_tfidf_bigrama = vectorizer.fit_transform(X_text)
X_tfidf_bigrama_norm = vectorizer.fit_transform(X_text_norm)

vectorizer = TfidfVectorizer(ngram_range=(3, 3))
X_tfidf_trigrama = vectorizer.fit_transform(X_text)
X_tfidf_trigrama_norm = vectorizer.fit_transform(X_text_norm)

In [7]:
# Vetorização com Bag of Words (BOW)
vectorizer = CountVectorizer(ngram_range=(1, 1))  # cria o vetor BOW
X_bow_unigrama = vectorizer.fit_transform(X_text)  # gera matriz esparsa
X_bow_unigrama_norm = vectorizer.fit_transform(X_text_norm)  # gera matriz esparsa normalizada

vectorizer = CountVectorizer(ngram_range=(2, 2))  # cria o vetor BOW
X_bow_bigrama = vectorizer.fit_transform(X_text)  # gera matriz esparsa
X_bow_bigrama_norm = vectorizer.fit_transform(X_text_norm)  # gera matriz esparsa normalizada

vectorizer = CountVectorizer(ngram_range=(3, 3))  # cria o vetor BOW
X_bow_trigrama = vectorizer.fit_transform(X_text)  # gera matriz esparsa
X_bow_trigrama_norm = vectorizer.fit_transform(X_text_norm)  # gera matriz esparsa normalizada

In [8]:
def sin_transform(X):
    return np.sin(X)

def cos_transform(X):
    return np.cos(X)

sin_transformer = FunctionTransformer(sin_transform, validate=True)
cos_transformer = FunctionTransformer(cos_transform, validate=True)

In [9]:
class DenseTransformer(TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return X.toarray() if hasattr(X, "toarray") else X

In [10]:

def build_classifier_pipeline(model_type='logistic', ensemble_method='voting'):
    # Modelos base individuais
    base_models = {
        'logistic': LogisticRegression(class_weight='balanced'),
        'decision_tree': DecisionTreeClassifier(class_weight='balanced'),
        'svm': SVC(class_weight='balanced',probability=True),
        'ridge': RidgeClassifier(class_weight='balanced')
    }
    
    # Definindo os ensembles
    ensembles = {
        'voting': VotingClassifier(
            estimators=[
                ('lr', LogisticRegression()),
                ('dt', DecisionTreeClassifier()),
                ('svm', SVC(probability=True))
            ],
            voting='soft'
        ),
        'stacking': StackingClassifier(
            estimators=[
                ('lr', LogisticRegression()),
                ('dt', DecisionTreeClassifier()),
                ('svm', SVC(probability=True))
            ],
            final_estimator=LogisticRegression()
        )
    }

    # Pipeline comum
    pipeline_steps = [
        ('scaler', StandardScaler())
    ]
    
    if model_type in ['voting', 'stacking']:
        pipeline_steps.append(('ensemble', ensembles[model_type]))
    else:
        pipeline_steps.append(('model', base_models[model_type]))
    
    return Pipeline(pipeline_steps)

In [22]:
def custom_param_grid(model_type):
    param_grid = {}
    
    # Modelos individuais
    if model_type == 'logistic':
        param_grid = {
            'model__C': [0.1, 1.0, 10.0],
            'model__penalty': ['l2', 'none'],
            'model__solver': ['lbfgs', 'saga'],  # Solvers que suportam multinomial
            'model__multi_class': ['multinomial'],  # Adicionado suporte multiclasse
            'model__max_iter': [500, 1000]  # Aumento para garantir convergência
        }
    
    elif model_type == 'decision_tree':
        param_grid = {
            'model__max_depth': [3, 5, None],
            'model__min_samples_split': [2, 5],
            'model__criterion': ['gini', 'entropy']  # Adicionado para multiclasse
        }
    
    elif model_type == 'svm':
        param_grid = {
            'model__C': [0.1, 1.0, 10.0],
            'model__kernel': ['linear', 'rbf'],
            'model__decision_function_shape': ['multinomial']  # Estratégia multiclasse
        }
    
    elif model_type == 'ridge':
        param_grid = {
            'model__alpha': [0.1, 1.0, 10.0],
            'model__solver': ['auto', 'saga']  # SAGA suporta multiclasse
        }
    
    # Ensembles (não requerem mudanças estruturais para multiclasse)
    elif model_type == 'voting':
        param_grid = {
            'ensemble__voting': ['soft', 'hard'],
            'ensemble__weights': [None, [1, 2, 1]]
        }
    
    elif model_type == 'stacking':
        param_grid = {
            'ensemble__final_estimator': [
                LogisticRegression(multi_class='multinomial', solver='lbfgs'),  # Adaptado
                DecisionTreeClassifier(max_depth=3)
            ]
        }
    
    return param_grid

In [24]:
from sklearn.metrics import balanced_accuracy_score, f1_score, roc_auc_score
from sklearn.exceptions import NotFittedError
import numpy as np
import gc

def outer_cv_evaluation(X, y, pipeline_fn, param_grid, outer_cv, inner_cv, chunk_size=10000):

    metrics = {
        'balanced_accuracy': [],
        'f1_score': [],
        # 'roc_auc': [],
        'support': []  # Número de amostras em cada fold
    }
    
    for fold_num, (train_idx, test_idx) in enumerate(outer_cv.split(X, y)):
        print(f"\nProcessando fold {fold_num + 1}/{outer_cv.get_n_splits()}")
        
        # Verificação de classes nos dados de treino
        unique_classes = np.unique(y[train_idx])
        if len(unique_classes) < 2:
            print(f"Aviso: Fold contém apenas uma classe ({unique_classes[0]}). Pulando...")
            for metric in metrics:
                metrics[metric].append(np.nan)
            continue
            
        # Processamento em chunks
        pipeline = pipeline_fn()
        n_chunks_train = int(np.ceil(len(train_idx) / chunk_size))
        model_fitted = False
        
        for i in range(n_chunks_train):
            start_idx = i * chunk_size
            end_idx = start_idx + chunk_size
            chunk_train_idx = train_idx[start_idx:end_idx]
            
            X_chunk = X[chunk_train_idx]
            y_chunk = y[chunk_train_idx]
            
            # Verificação de classes no chunk atual
            if len(np.unique(y_chunk)) < 2:
                continue
                
            if hasattr(X_chunk, "toarray"):
                X_chunk = X_chunk.toarray() if X_chunk.nnz < (0.1 * X_chunk.shape[0] * X_chunk.shape[1]) else X_chunk
            
            try:
                if hasattr(pipeline, 'partial_fit'):
                    pipeline.partial_fit(X_chunk, y_chunk, classes=unique_classes)
                else:
                    pipeline.fit(X_chunk, y_chunk)
                model_fitted = True
            except ValueError as e:
                print(f"Erro no treino do chunk {i+1}: {str(e)}")
                continue

        # Avaliação
        test_chunks = [test_idx[i:i+chunk_size] for i in range(0, len(test_idx), chunk_size)]
        y_preds, y_probs, y_tests = [], [], []
        
        for chunk in test_chunks:
            X_test_chunk = X[chunk]
            if hasattr(X_test_chunk, "toarray"):
                X_test_chunk = X_test_chunk.toarray() if X_test_chunk.nnz < (0.1 * X_test_chunk.shape[0] * X_test_chunk.shape[1]) else X_test_chunk
            
            try:
                if model_fitted:
                    y_preds.extend(pipeline.predict(X_test_chunk))
                    # Alguns modelos podem não ter predict_proba
                    if hasattr(pipeline, 'predict_proba'):
                        y_probs.extend(pipeline.predict_proba(X_test_chunk)[:, 1])
                    y_tests.extend(y[chunk])
            except NotFittedError:
                print(f"Aviso: Modelo não treinado no fold {fold_num+1}")
                y_preds.extend([np.nan] * len(chunk))
                y_tests.extend(y[chunk])

        # Cálculo das métricas
        if len(y_tests) > 0 and not all(np.isnan(y_preds)):
            y_tests = np.array(y_tests)
            y_preds = np.array(y_preds)
            
            metrics['balanced_accuracy'].append(balanced_accuracy_score(y_tests, y_preds))
            metrics['f1_score'].append(f1_score(y_tests, y_preds, average='weighted'))
            
            # if len(y_probs) > 0:
            #     metrics['roc_auc'].append(roc_auc_score(y_tests, y_probs, multi_class='ovo', average='weighted'))
            # else:
            #     metrics['roc_auc'].append(np.nan)
                
            metrics['support'].append(len(y_tests))
        else:
            for metric in metrics:
                metrics[metric].append(np.nan)
        
        # Liberação de memória
        del X_chunk, y_chunk, X_test_chunk, pipeline
        gc.collect()

    return metrics

In [13]:
X_combined_tfidf_norm = [X_tfidf_unigrama_norm, X_tfidf_bigrama_norm, X_tfidf_trigrama_norm]
X_combined_tfidf = [X_tfidf_unigrama, X_tfidf_bigrama, X_tfidf_trigrama]
X_combined_bow_norm = [X_bow_unigrama_norm, X_bow_bigrama_norm, X_bow_trigrama_norm]
X_combined_bow = [X_bow_unigrama, X_bow_bigrama, X_bow_trigrama]

X_combined_bow = [X_combined_bow_norm, X_combined_bow]
X_combined_tfidf = [X_combined_tfidf_norm, X_combined_tfidf]

X_combined_all = [X_combined_bow, X_combined_tfidf]

In [14]:
def analyze_cv_results(results):
    """Analisa os resultados da validação cruzada de forma robusta"""
    print("\n=== Análise Detalhada ===")
    
    metric_stats = {}
    
    for metric, values in results.items():
        # Filtra valores não numéricos ou inválidos
        valid_values = []
        for v in values:
            try:
                # Tenta converter para float e verifica se é numérico válido
                v_float = float(v)
                if not np.isnan(v_float):
                    valid_values.append(v_float)
            except (TypeError, ValueError):
                continue
        
        # Calcula estatísticas apenas se houver valores válidos
        if valid_values:
            metric_stats[metric] = {
                'mean': np.mean(valid_values),
                'std': np.std(valid_values),
                'n_folds': len(valid_values),
                'min': np.min(valid_values),
                'max': np.max(valid_values)
            }
    
    # Exibe os resultados formatados
    for metric, stats in metric_stats.items():
        print(f"\n{metric.upper()}:")
        print(f"  Média: {stats['mean']:.4f}")
        print(f"  Desvio Padrão: {stats['std']:.4f}")
        print(f"  Folds válidos: {stats['n_folds']}/{len(results[metric])}")
        print(f"  Variação: [{stats['min']:.4f}, {stats['max']:.4f}]")
    
    return metric_stats

In [ ]:
from imblearn.combine import SMOTETomek
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE

outer_cv = KFold(n_splits=2, shuffle=True, random_state=42)
inner_cv = KFold(n_splits=2, shuffle=True, random_state=42)

cont2 = 0

# Avaliação
for x_combined_1 in X_combined_all:
    cont2 += 1
    print(f"\n\nAvaliação com {'BOW' if cont2 == 1 else 'TF-IDF'}:\n")
    cont3 = 0
    for x_combined in x_combined_1:
        cont3 += 1
        if( cont3 == 1):
                print("Com pré-proecessamento:")
        elif( cont3 == 2):
            print("Sem pré-processamento:")
        cont = 0
        for X in x_combined:
            cont += 1
            if cont == 1:
                print(" Unigrama:")
            elif cont == 2:
                print(" Bigrama:")
            elif cont == 3:
                print(" Trigrama:")
            results = {}
            for model in ['logistic', 'decision_tree', 'svm', 'ridge', 'voting', 'stacking']:                
                
                scores = outer_cv_evaluation(
                    X, y,
                    pipeline_fn=lambda: build_classifier_pipeline(model_type=model),
                    param_grid=custom_param_grid(model),
                    outer_cv=outer_cv,
                    inner_cv=inner_cv,
                    chunk_size=500  # Ajuste conforme necessário
                )
                
                # Análise robusta dos resultados
                metric_stats = analyze_cv_results(scores)                
                
                results[model] = scores



Avaliação com BOW:

Com pré-proecessamento:
 Unigrama:

Processando fold 1/2


/home/raylander/Desktop/Introdução a Ciência de Dados/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/raylander/Desktop/Introdução a Ciência de Dados/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:


Processando fold 2/2


/home/raylander/Desktop/Introdução a Ciência de Dados/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/raylander/Desktop/Introdução a Ciência de Dados/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:


=== Análise Detalhada ===

BALANCED_ACCURACY:
  Média: 0.3439
  Desvio Padrão: 0.0085
  Folds válidos: 2/2
  Variação: [0.3355, 0.3524]

F1_SCORE:
  Média: 0.3218
  Desvio Padrão: 0.0216
  Folds válidos: 2/2
  Variação: [0.3001, 0.3434]

SUPPORT:
  Média: 106952.0000
  Desvio Padrão: 0.0000
  Folds válidos: 2/2
  Variação: [106952.0000, 106952.0000]

Processando fold 1/2

Processando fold 2/2

=== Análise Detalhada ===

BALANCED_ACCURACY:
  Média: 0.3429
  Desvio Padrão: 0.0028
  Folds válidos: 2/2
  Variação: [0.3401, 0.3457]

F1_SCORE:
  Média: 0.3500
  Desvio Padrão: 0.0121
  Folds válidos: 2/2
  Variação: [0.3378, 0.3621]

SUPPORT:
  Média: 106952.0000
  Desvio Padrão: 0.0000
  Folds válidos: 2/2
  Variação: [106952.0000, 106952.0000]

Processando fold 1/2
